In deze opgave classificeren afbeeldingen van bloemen uit de iris dataset met behulp van een neuraal netwerk.

In [1]:
import numpy as np
import pandas as pd
from matplotlib import pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import OneHotEncoder

Laad de dataset en maak een feature matrix X en een target vector y.

In [2]:
iris = sns.load_dataset('iris')

# we gebruiken .values om een Numpy array te krijgen in plaats van een Pandas DataFrame
X_iris = iris.drop('species', axis=1).values 
y_iris = iris['species'].values

In [3]:
y_iris = y_iris.reshape(-1, 1) # maak een array van array's, dit is nodig voor de volgende stap
print(y_iris[:10])

[['setosa']
 ['setosa']
 ['setosa']
 ['setosa']
 ['setosa']
 ['setosa']
 ['setosa']
 ['setosa']
 ['setosa']
 ['setosa']]


Een neuraal netwerk kan niet omgaan met categorische labels zoals de namen van de planten, maar heeft getallen als uitput nodig. We hebben hier drie klassen (setosa, versicolor, virginica) daarom maken we een target vector met per label drie waarden. Een 1 voor de eerste waarde correspondeert met 'setosa', een 1 voor de 2e waade met 'versicolor' en een 1 voor de derde waarde met 'virginica'.

Doe dit met behulp van het `OneHotEncoder` object uit sklearn.

In [4]:
enc = OneHotEncoder(sparse=False)
y_enc = enc.fit_transform(y_iris)

We hebben de data nu in een geschikt formaat. Splits de data in training en test data.

In [5]:
X_train, X_test, y_train, y_test = train_test_split(X_iris, y_enc, test_size=0.2, random_state=6)

Maak een eenvoudig neuraal netwerk met 3 lagen, waarin elke neuron in een laag met elk neuron in de vorige laag verbonden is ('Dense'). 
Voeg voor de eeste laag een parameter `input_shape` toe die aangeeft hoeveel features een element in 'X' heeft.
Het aantal neuronen in de laaste laag is gelijk aan het aantal waarden in een label in `y`. 

In [6]:
from keras.models import Sequential
from keras.layers import Dense

model = Sequential()

# Voeg lagen toe
model.add(Dense(4, input_shape = (4,), activation = 'relu'))
model.add(Dense(3, activation= "relu"))
model.add(Dense(3, activation = 'softmax'))

Compileer en bekijk het netwerk door onderstaande code te runnen.

In [7]:
from keras.optimizers import Adam

model.compile(Adam(lr=0.01),'categorical_crossentropy',metrics=['accuracy'])

model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense (Dense)               (None, 4)                 20        
                                                                 
 dense_1 (Dense)             (None, 3)                 15        
                                                                 
 dense_2 (Dense)             (None, 3)                 12        
                                                                 
Total params: 47
Trainable params: 47
Non-trainable params: 0
_________________________________________________________________


C:\Users\misst\aai-env\lib\site-packages\keras\optimizers\optimizer_v2\adam.py:110: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  super(Adam, self).__init__(name, **kwargs)


Nu kunnen we ons model trainen met behulp van de `fit` methode.

In [8]:
model.fit(X_train, y_train, epochs = 25, batch_size = 5)

Epoch 1/25
24/24 [==============================] - 1s 1ms/step - loss: 0.9707 - accuracy: 0.5333 
Epoch 2/25
24/24 [==============================] - 0s 1ms/step - loss: 0.7582 - accuracy: 0.6667
Epoch 3/25
24/24 [==============================] - 0s 1ms/step - loss: 0.6162 - accuracy: 0.7167
Epoch 4/25
24/24 [==============================] - 0s 1ms/step - loss: 0.4968 - accuracy: 0.7667
Epoch 5/25
24/24 [==============================] - 0s 1ms/step - loss: 0.4287 - accuracy: 0.8333
Epoch 6/25
24/24 [==============================] - 0s 1ms/step - loss: 0.3749 - accuracy: 0.8833
Epoch 7/25
24/24 [==============================] - 0s 1ms/step - loss: 0.3239 - accuracy: 0.9667
Epoch 8/25
24/24 [==============================] - 0s 1ms/step - loss: 0.2886 - accuracy: 0.8833
Epoch 9/25
24/24 [==============================] - 0s 1ms/step - loss: 0.2822 - accuracy: 0.9000
Epoch 10/25
24/24 [==============================] - 0s 2ms/step - loss: 0.2271 - accuracy: 0.9500
Epoch 11/25
24/24 

Met `model_evaluate` kunnen we bepalen hoe goed het model werkt op de test data.

In [9]:
test_loss, test_acc = model.evaluate(X_test, y_test)
print(test_acc)

1/1 [==============================] - 0s 240ms/step - loss: 0.0734 - accuracy: 0.9667
0.9666666388511658


Experimenteer met bovenstaand model. Wat gebeurt er als je meer lagen toevoegt of een laag weghaalt of het aantal neuronen in een laag verandert? 

Probeer ook eens andere activatiefuncties dan `relu`, zoals `tanh` of `sigmoid`.

Welke invloed heeft het aantal epochs?

###### Bij het toevoegen van lagen krijg ik een hogere accuracy terug. Hetzelfde gelf voor als er lagen worden weggehaald, ik krijg dan een lagere accuracy terug.

We kunnen dezelfde aanpak gebruiken voor andere data, bijvoorbeeld voor het herkennen van cijfers. We gebruiken de `MNist` dataset die bestaat uit plaatjes van cijfers. Ieder plaatje bestaat uit 28 x 28 pixels.

In [10]:
from keras.datasets import mnist

(train_images, train_labels), (test_images, test_labels) = mnist.load_data()

import warnings
warnings.filterwarnings('ignore')

In [11]:
train_images.shape

(60000, 28, 28)

In [12]:
len(train_labels)

60000

In [13]:
print(train_labels)

[5 0 4 ... 5 6 8]


Een eenvoudig neuraal netwerk heeft een array van inputwaarden nodig i.p.v. een 2D afbeelding.
We maken van de inputs (waarden tussen 0 en 255) getallen tussen 0 en 1.

In [14]:
train_images = train_images.reshape((60000, 28 * 28)) # lijst van waarden i.p.v. 2D afbeelding
train_images = train_images.astype('float32') / 255 # getallen tussen 0 en 1

test_images = test_images.reshape((10000, 28 * 28))
test_images = test_images.astype('float32') / 255

Voor elk label (een cijfer) maken we een array met 10 waarden: 9 nullen en een één, waarbij de positie van de één aangeeft om welk cijfer het gaat (One hot encoding). Dit keer gebruiken we hiervoor de `keras` functie `to_categorical`.

In [15]:
from keras.utils import to_categorical

train_labels = to_categorical(train_labels) 
test_labels = to_categorical(test_labels)

print(train_labels)

[[0. 0. 0. ... 0. 0. 0.]
 [1. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 ...
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 1. 0.]]


Maak nu een neuraal netwerk met twee lagen, een eerste laag met 512 neuronen en een output laag met 10 neuronen. Geef bij de eerste laag aan hoeveel inputs er zijn en wat de activatiefunctie is. De output laag heeft een `softmax` activatiefunctie.

In [16]:
network = Sequential()
network.add(Dense(512, activation = 'relu', input_shape= (28 * 28, ))
)
network.add(Dense(10, activation = 'softmax'))

compileer het netwerk.

In [17]:
network.compile(Adam(lr=0.01),'categorical_crossentropy',metrics=['accuracy'])

train het netwerk 5 epochs met een batch_size van 128.

In [18]:
network.fit(train_images, train_labels, epochs = 5, batch_size = 128) 

Epoch 1/5
469/469 [==============================] - 7s 6ms/step - loss: 0.2117 - accuracy: 0.9360
Epoch 2/5
469/469 [==============================] - 3s 6ms/step - loss: 0.1097 - accuracy: 0.9669
Epoch 3/5
469/469 [==============================] - 3s 6ms/step - loss: 0.0915 - accuracy: 0.9725
Epoch 4/5
469/469 [==============================] - 3s 6ms/step - loss: 0.0754 - accuracy: 0.9773
Epoch 5/5
469/469 [==============================] - 3s 6ms/step - loss: 0.0732 - accuracy: 0.9794


Evalueer het netwerk op de test set.

In [20]:
test_loss, test_acc = network.evaluate(test_images, test_labels)
print(test_acc)

313/313 [==============================] - 3s 2ms/step - loss: 0.1250 - accuracy: 0.9712
0.9711999893188477


Experimenteer met verschillende netwerken. Varieer het aantal lagen, het aantal neuronen, de activatiefunties en het aantal epochs.